In [4]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd

from numba import jit

### Introdução
A atividade a ser explorada neste trabalho diz respeito ao processo de termalização. Aqui será apresentada uma simulação do modelo de Ising 2D com base no algoritmo de Metropolis. 
O modelo de Ising 2D pode ser entendido como um grid de pontos, no qual cada ponto representa um spin, isto é, um momento magnético. Cada spin pode assumir dois valores: +1 ou -1. O modelo de Ising 2D é um modelo de mecânica estatística que descreve a interação entre spins vizinhos.

In [2]:
def initial_system_config(N):
    """
    Define a configuração inicial do sistema aleatoriamente 
    S é um array unidimensional que recebe valores aleatórios -1 ou 1 em cada uma de suas posições
    """
    S = np.random.randint(0,2,N)
    S = 2 * S - 1
    return S

Para fins de otimização, optou-se por representar o grid do modelo de ising como um array unidemensional. Para isso, foi necessário criar uma função que mapeia a posição de um spin no grid para a posição correspondente no array. Essa função é apresentada abaixo.

In [3]:
@jit(nopython=True)
def vizinhos(N):
    """
    Define a tabela de vizinhos
    """
    L=int(np.sqrt(N))
    viz = np.zeros((N,4),dtype=np.int16)
    for k in range(N):
        viz[k,0]=k+1
        if (k+1) % L == 0:
            viz[k,0] = k+1-L
        viz[k,1] = k+L
        if k > (N-L-1):
            viz[k,1] = k+L-N
        viz[k,2] = k-1
        if (k % L == 0):
            viz[k,2] = k+L-1
        viz[k,3] = k-L
        if k < L:
            viz[k,3] = k+N-L
    return viz

Seguindo com otimizações, aqui temos uma função que define um vetor de 5 posições de acordo com as possibilidades de valor para a diferença de energia entre dois estados. Essa função é utilizada para evitar o cálculo da exponencial de um número muito grande e muitas vezes, armazenanado os valores possíveis em um vetor e acessando-os quando necessário.

In [4]:
def expos(beta):
    ex = np.zeros(5,dtype=np.float32)
    ex[0]=np.exp(8.0*beta)
    ex[1]=np.exp(4.0*beta)
    ex[2]=1.0
    ex[3]=np.exp(-4.0*beta)
    ex[4]=np.exp(-8.0*beta)
    return ex

Em seguida é a presentada a função que calcula a energia do sistema, a qual é utilizada para calcular a energia do sistema antes e depois de uma mudança de spin(flip). A diferença entre esses dois valores é utilizada para calcular a probabilidade de aceitação da mudança de spin. Aprofundaremos esses conceitos mais adiante.

In [5]:
@jit(nopython=True)
def energy(N, viz, S):
    E = 0.0
    for i in range(N):
        h = S[viz[i,0]] + S[viz[i,1]]
        E += -S[i]*h
    return E

Aqui temos o algoritmo de Metropolis, que é utilizado para simular o modelo de Ising 2D. O algoritmo consiste em um loop que itera sobre cada spin do grid. Para cada spin, é calculada a energia do sistema antes e depois de um flip. A diferença entre esses dois valores é utilizada para calcular a probabilidade de aceitação da mudança de spin. Caso a probabilidade de aceitação seja maior que um número aleatório entre 0 e 1, o flip é aceito. Caso contrário, o flip é rejeitado. Esse processo é repetido até que o sistema atinja o equilíbrio.

In [6]:
def metroplis_ising(N, T, mc_steps):    
    beta = 1/T
    ex = expos(beta)
    S = initial_system_config(N)
    energy_system = []
    magnetizations = []
    viz = vizinhos(N)
    
    for _ in range(mc_steps):
        for spin in range(N):
            h = S[viz[spin,0]] + S[viz[spin,1]] + S[viz[spin,2]] + S[viz[spin,3]] # soma dos vizinhos
            de = int(S[spin] * h * 0.5 + 2)
            if random.random() < ex[de]:  # random.random gera um número aleatório entre 0 e 1
                S[spin] = -S[spin]
        energy_system.append(energy(N, viz, S))   
        magnetizations.append(np.sum(S)) 
    
    return energy_system, magnetizations

# Temperatura está em unidades de j/kB, onde kb é a constante de Boltzmann

As duas funções abaixo são usadas para plotar tanto os estados de energia quanto a magnetização do sistema. O eixo y representa a energia ou a magnetização, enquanto o eixo x representa o número de iterações (passos de Monte Carlo).

In [7]:
def _plot(values, mc_steps, N, T, xlabel='Monte Carlo Step', ylabel='Energy', title='Energy vs MC steps'):
    for value in values:
        plt.plot(mc_steps, value)

    title = f'{title} (N={N}, T={T})'
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

In [8]:
def plot_systems_energy(systems, magnetizations, N, T, mc_steps=1000):
    mc_steps = np.arange(0, mc_steps, 1)
    _plot(systems, mc_steps, N, T)
    _plot(magnetizations, mc_steps, N, T, ylabel='Magnetization', title='Magnetization vs MC steps')

Abaixo encontram-se as funções que manejam o algoritmo de Metropolis. Elas recebem como parâmetros o estado inicial do sistema, o número de iterações (passos de Monte Carlo) e a temperatura. Ela retorna o número de spins, temperatura, energia e magnetização definidos para o sistema a partir da chamada de metropolis_ising. Os dois últimos encontram-se em listas, nas quais cada uma das posições dizem respeito ao estado do sistema após um passo de monte carlo.

Aqui optou-se por permitir ao usuário que escolha entre a configuração padrão - definida pelo enunciado da atividade - ou uma configuração aleatória. O controlador disso é o parâmetro standard, que por padrão é True, definindo 1024 spins e temperatura = 1.5. Caso o usuário queira uma configuração aleatória, basta passar o parâmetro como False e então serão gerados N e T em um intervalo definido T = [0.4, 3] real  e N = [20, 100] inteiro.

In [9]:
def generate_systems(N=1024, T=1.5, n_systems=10,  mc_steps=1000):
    """
    Função para gerar sistemas A partir de valores definidos de Número de spins(N) e Temperatura (T). \n
    O caso de padrão tem temperatura é fixa em 1.5 e o tamanho de cada sistema em 1024 spins.
    """
    energies = []
    magnetizations = []

    for _ in range(n_systems):
        energy, magnetization = metroplis_ising(N, T, mc_steps)
        energies.append(energy)
        magnetizations.append(magnetization)
        
    return energies, magnetizations, N, T

In [10]:
def generate_random_systems(L_range=[24,100], T_range=[0.4, 3], n_systems=10,  mc_steps=1000):
    """
    Função para gerar sistemas aleatórios \n
    No caso de aleatórios a temperatura e o tamanho do sistema são gerados aleatoriamente
    dentro dos intervalos definidos por T_range e N_range, respectivamente, que são listas de dois valores.
    """
    energies = []
    magnetizations = []
    
    T = np.random.uniform(T_range[0], T_range[1])
    N = np.random.randint(L_range[0], L_range[1]) ** 2

    for _ in range(n_systems):
        energy, magnetization = metroplis_ising(N, T, mc_steps)
        energies.append(energy)
        magnetizations.append(magnetization)
        
    return energies, magnetizations, N, T

In [ ]:
def specific_heat(N=1024, T=1.5, energies=None, mc_steps=1000):
    """
    Calculates the specific heat of the system
    """
    if energies is None:
        energies, _, _, _ = generate_systems(N, T, 1, mc_steps)
    
    energies = np.array(energies)
    energy_mean = np.mean(energies)
    energy_mean2 = np.mean(energies**2)
    
    specific_heat = (energy_mean2 - energy_mean**2) * ((1/T**2) / N)

    return specific_heat

In [ ]:
def magnetic_susceptibility(N=1024, T=1.5, magnetizations=None, mc_steps=1000):
    """
    Calculates the magnetic susceptibility of the system
    """
    if magnetizations is None:
        _, magnetizations, _, _ = generate_systems(N, T, 1, mc_steps)
    
    magnetizations = np.array(magnetizations)
    magnetizations = np.absolute(magnetizations)
    magnetization_mean = np.mean(magnetizations)
    magnetization_mean2 = np.mean(magnetizations**2)
    
    magnetic_susceptibility = (magnetization_mean2 - magnetization_mean**2) * ((1/T) / N)

    return magnetic_susceptibility

In [ ]:
def energyPerSpin(energyList, size):
    energyPerSpin = np.mean(energyList)/(size*size)
    return energyPerSpin

In [ ]:
def magnetizationPerSpin(magnetizationList, size):
    magnetizationPerSpin = np.mean(magnetizationList)/(size*size)
    return magnetizationPerSpin

In [13]:
values_list = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
n_blocks = 5

boxes = np.array_split(values_list, n_blocks)
values_estimate = [[np.mean(box) for box in boxes]]
print(boxes, values_estimate)

[array([1, 2]), array([3, 4]), array([5, 6]), array([7, 8]), array([ 9, 10])] [[1.5, 3.5, 5.5, 7.5, 9.5]]


In [16]:
def calc_error(mean_value, boxes):
    """
    Calculates the statistical error of the values
    """
    return np.sqrt(np.sum((np.repeat(mean_value, len(boxes)) - boxes)**2) / len(boxes) * (len(boxes) - 1))

In [ ]:
def statistical_error_boxes_method(n_blocks, values_list, N, T, mc_steps=1000):
    boxes = np.array_split(values_list, n_blocks)
    values_estimate = [np.mean(box) for box in boxes]
    
    sh = [specific_heat(N, T, value, mc_steps) for value in values_estimate]
    sh = np.mean(sh)
    
    return calc_error(sh, values_estimate)